In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
import tensorflow as tf



In [2]:
data_train = pd.read_csv("train_dataset_train.csv")

In [3]:
data_train.head(3)

,id,Easting,Northing,Height,Reflectance,Class
0,2321251,431696.5375,6.032319e+06,69.2226,-11.14,0
1,3515173,431710.3835,6.032291e+06,68.9711,-15.16,3
2,2320295,431696.8099,6.032322e+06,69.2453,-13.59,0


In [4]:
data_y = data_train["Class"]
encod_y = LabelBinarizer()
Y_train = encod_y.fit_transform(data_y)

In [5]:
def get_train_dataset_x(data_x):

  data_X = data_x.copy()
  norm = MinMaxScaler()
  pca = PCA(5)
  res = norm.fit_transform(data_X)
  res = pca.fit_transform(res)
  X_train = res

  return X_train, norm, pca

In [6]:
def get_test_dataset_x(data_x, norm, pca):

  data_X = data_x.copy()
  res = norm.transform(data_X)
  res = pca.transform(res)

  X_test = res

  return X_test

In [7]:
data_x_best = data_train[data_train.columns[:-1]]
X_train, norm, pca = get_train_dataset_x(data_x_best)

In [21]:
best_model = tf.keras.models.load_model("model_best.h5")

In [9]:
best_model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_27 (Dense)            (None, 1000)              6000      
                                                                 
 dense_28 (Dense)            (None, 1000)              1001000   
                                                                 
 dense_29 (Dense)            (None, 1000)              1001000   
                                                                 
 dense_30 (Dense)            (None, 1000)              1001000   
                                                                 
 dense_31 (Dense)            (None, 1000)              1001000   
                                                                 
 dense_32 (Dense)            (None, 6)                 6006      
                                                                 
Total params: 4,016,006
Trainable params: 4,016,006
No

In [55]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Recall
from tensorflow.keras.optimizers.schedules import ExponentialDecay
initial_learning_rate = 0.00000001
lr_schedule = ExponentialDecay(
    initial_learning_rate,
    decay_steps=1000000,
    decay_rate=1,
    staircase=True)
opt = Adam(learning_rate=lr_schedule)
met = Recall()
best_model.compile(optimizer=opt, loss="categorical_crossentropy", metrics=[met])

In [56]:
history_best = best_model.fit(X_train, Y_train, epochs=100, validation_split = 0.01, batch_size=40000, verbose = 2)

Epoch 1/100
105/105 - 26s - loss: 0.0173 - recall_19: 0.9929 - val_loss: 0.0291 - val_recall_19: 0.9883 - 26s/epoch - 250ms/step
Epoch 2/100
105/105 - 25s - loss: 0.0173 - recall_19: 0.9929 - val_loss: 0.0291 - val_recall_19: 0.9883 - 25s/epoch - 240ms/step
Epoch 3/100
105/105 - 25s - loss: 0.0173 - recall_19: 0.9929 - val_loss: 0.0291 - val_recall_19: 0.9883 - 25s/epoch - 240ms/step
Epoch 4/100
105/105 - 25s - loss: 0.0173 - recall_19: 0.9929 - val_loss: 0.0291 - val_recall_19: 0.9883 - 25s/epoch - 241ms/step
Epoch 5/100
105/105 - 25s - loss: 0.0173 - recall_19: 0.9929 - val_loss: 0.0291 - val_recall_19: 0.9883 - 25s/epoch - 241ms/step
Epoch 6/100
105/105 - 25s - loss: 0.0173 - recall_19: 0.9929 - val_loss: 0.0291 - val_recall_19: 0.9883 - 25s/epoch - 241ms/step
Epoch 7/100
105/105 - 25s - loss: 0.0172 - recall_19: 0.9929 - val_loss: 0.0291 - val_recall_19: 0.9883 - 25s/epoch - 241ms/step
Epoch 8/100
105/105 - 25s - loss: 0.0172 - recall_19: 0.9929 - val_loss: 0.0291 - val_recall_19: 

In [ ]:
best_model.save(f"model_best.h5")

In [16]:
plt.plot(history_best.history["loss"])

NameError: name 'history_best' is not defined

In [57]:
test = pd.read_csv("test_dataset_test.csv")
X_test = get_test_dataset_x(test[test.columns[:]], norm, pca)
Y_test = best_model.predict(X_test, batch_size = 100000, verbose=1)

19/19 [==============================] - 4s 236ms/step


In [58]:
classif = encod_y.inverse_transform(Y_test)
id = test["id"].to_numpy()
res = list(zip(id,classif))
out = pd.DataFrame(res, columns=["id", "Class"])
out.to_csv(f"result_13.csv", index=False)


